### 5. Feature Engineering

In [1]:
# import libraries used

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import  OneHotEncoder, LabelEncoder

import swifter
import gc

In [2]:
# Load Processed Data

# load categories
df_cats = pd.read_csv("v30_item_categories_clean.csv")
# load items
df_items = pd.read_csv("v30_items_clean.csv")
# load transaction data for trainning
df_train = pd.read_csv("v30_sales_train_clean.csv")
# load test data set
df_test = pd.read_csv("v30_test_clean.csv")
# load shops
df_shops = pd.read_csv("v30_shops_clean.csv")

#### 5.1 Trainning Data Framework Preparation

In [3]:
#shop_item_pair = list(df_train[['shop_id', 'item_id']].drop_duplicates().values)

In [4]:
from itertools import product

df_train_x = []
index_cols = ['date_block_num','shop_id','item_id']

for i in range(34):
    #np_date = np.array([i]*len(shop_item_pair)).reshape(-1, 1)
    #df_train_x.append(np.array(list(np.hstack((np_date, shop_item_pair))), dtype='int16'))
    sales = df_train[df_train.date_block_num == i]
    df_train_x.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))


    
df_train_x = pd.DataFrame(np.vstack(df_train_x), columns=index_cols)
df_train_x['date_block_num'] = df_train_x['date_block_num'].astype(np.int8)
df_train_x['shop_id'] = df_train_x['shop_id'].astype(np.int8)
df_train_x['item_id'] = df_train_x['item_id'].astype(np.int16)

df_train_x.sort_values(index_cols, inplace=True)
df_train_x

,date_block_num,shop_id,item_id
317785,0,2,19
320025,0,2,27
323498,0,2,28
321191,0,2,29
317477,0,2,32
...,...,...,...
10823247,33,59,22162
10825275,33,59,22163
10824330,33,59,22164
10822444,33,59,22166


In [5]:
del df_test['ID']

In [6]:
df_test['date_block_num'] = 34
df_test['date_block_num'] = df_test['date_block_num'].astype(np.int8)
df_test['shop_id'] = df_test['shop_id'].astype(np.int8)
df_test['item_id'] = df_test['item_id'].astype(np.int16)
df_test

,shop_id,item_id,date_block_num
0,5,5037,34
1,5,5320,34
2,5,5233,34
3,5,5232,34
4,5,5268,34
...,...,...,...
214195,45,18454,34
214196,45,16188,34
214197,45,15757,34
214198,45,19648,34


In [7]:
df_train_x = pd.concat([df_train_x, df_test], ignore_index=True, sort=False, keys=index_cols)
df_train_x

,date_block_num,shop_id,item_id
0,0,2,19
1,0,2,27
2,0,2,28
3,0,2,29
4,0,2,32
...,...,...,...
11126961,34,45,18454
11126962,34,45,16188
11126963,34,45,15757
11126964,34,45,19648


#### 5.2 Basic Features

In [8]:
# extract and correct the category type info.
df_cats['cat_type'] = LabelEncoder().fit_transform(df_cats['cat_type'])
df_cats['sub_type'] = LabelEncoder().fit_transform(df_cats['sub_type'])
df_cats = df_cats[['item_category_id','cat_type', 'sub_type']]

df_cats

,item_category_id,cat_type,sub_type
0,0,0,11
1,1,0,12
2,2,0,13
3,3,0,14
4,4,0,16
...,...,...,...
79,79,12,56
80,80,12,28
81,81,13,65
82,82,13,66


In [9]:
df_shops['shop_city'] = LabelEncoder().fit_transform(df_shops['shop_city'])
df_shops['shop_type'] = LabelEncoder().fit_transform(df_shops['shop_type'])

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

feature_count = 25
tfidf = TfidfVectorizer(max_features=feature_count)
df_item_name_text_features = pd.DataFrame(tfidf.fit_transform(df_items['item_name']).toarray())

cols = df_item_name_text_features.columns
for i in range(feature_count):
    feature_name = 'item_name_tfidf_' + str(i)
    df_items[feature_name] = df_item_name_text_features[cols[i]]

df_items.drop('item_name', axis = 1, inplace = True)

In [11]:
df_train_x = pd.merge(df_train_x, df_shops, on=['shop_id'], how='left')
df_train_x = pd.merge(df_train_x, df_items, on=['item_id'], how='left')
df_train_x = pd.merge(df_train_x, df_cats, on=['item_category_id'], how='left')

df_train_x.drop('shop_name', axis=1, inplace=True)
df_train_x['shop_city'] = df_train_x['shop_city'].astype(np.int8)
df_train_x['shop_type'] = df_train_x['shop_type'].astype(np.int8)
df_train_x['cat_type'] = df_train_x['cat_type'].astype(np.int8)
df_train_x['sub_type'] = df_train_x['sub_type'].astype(np.int8)
df_train_x['item_id'] = df_train_x['item_id'].astype(np.int16)
df_train_x['item_category_id'] = df_train_x['item_category_id'].astype(np.int8)

In [12]:
#calculate the time-to-market for each item
#34 is the invalid value, for there are only 33 months's data.

time_to_market = [34]*(df_items.item_id.max()+1)

for i in range(34):
    sales = df_train[df_train.date_block_num == i]
    items = sales.item_id.unique()
    for item in items:
        if time_to_market[item] == 34:
            time_to_market[item] = i

In [13]:
def cal_time_on_market(row):
    return row.date_block_num - time_to_market[int(row.item_id)]

In [14]:
# generate time_on_market feature.
df_train_x['time_on_market'] = df_train_x.swifter.apply(cal_time_on_market, axis=1).astype(np.int8)
df_train_x

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_18,item_name_tfidf_19,item_name_tfidf_20,item_name_tfidf_21,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,7,6,0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.578850,0.0,0.0,0.0,1,13,0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.618795,0.0,0.0,0.0,4,57,0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.000000,0.0,0.468373,0.0,0.0,0.0,1,19,0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,7,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,9,4,11
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,10,44,2
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0,9,4,34
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,1.0,0.000000,0.0,0.0,0.0,7,6,11


In [15]:
'''
drop_list = df_train_x.loc[df_train_x.time_on_market < 0].index
df_train_x.drop(drop_list, inplace=True)
df_train_x
'''

'\ndrop_list = df_train_x.loc[df_train_x.time_on_market < 0].index\ndf_train_x.drop(drop_list, inplace=True)\ndf_train_x\n'

In [16]:
date_num = 35
item_num = (df_train.item_id.max() + 1)
item_no_sale = [-1] * date_num * item_num
all_items = df_train_x.item_id.sort_values().unique()

for date in range(1, 35):  
    print("date {}".format(date))
    date_sales = df_train[df_train.date_block_num == (date - 1)].reset_index(drop=True)
    date_items = date_sales.item_id.unique()
    for item in all_items:
        if (item not in date_items):
            if item_no_sale[(date - 1) * item_num + item] != -1:
                item_no_sale[date * item_num + item] = item_no_sale[(date - 1) * item_num + item] + 1    
        else:
            item_no_sale[date * item_num + item] = 0
        
def apply_item_no_sale(row):
    return item_no_sale[int(row.date_block_num) * item_num + int(row.item_id)]

df_train_x['time_no_sale'] = df_train_x.swifter.apply(apply_item_no_sale, axis=1).astype(np.int8)
df_train_x

date 1
date 2
date 3
date 4
date 5
date 6
date 7
date 8
date 9
date 10
date 11
date 12
date 13
date 14
date 15
date 16
date 17
date 18
date 19
date 20
date 21
date 22
date 23
date 24
date 25
date 26
date 27
date 28
date 29
date 30
date 31
date 32
date 33
date 34


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_19,item_name_tfidf_20,item_name_tfidf_21,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,7,6,0,-1
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.578850,0.0,0.0,0.0,1,13,0,-1
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.618795,0.0,0.0,0.0,4,57,0,-1
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.000000,0.0,0.468373,0.0,0.0,0.0,1,19,0,-1
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,7,6,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,9,4,11,0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,10,44,2,0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,9,4,34,0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,1.0,0.000000,0.0,0.0,0.0,7,6,11,0


In [17]:
shop_to_market = [34]*(df_train.shop_id.max() + 1)

for shop in df_train.shop_id.unique():
    date_launch = df_train[df_train.shop_id == shop].date_block_num.min()
    if not pd.isnull(date_launch):
        shop_to_market[shop] = date_launch

def apply_shop_on_market(row):
    return row.date_block_num - shop_to_market[int(row.shop_id)]
    
df_train_x['shop_on_market'] = df_train_x.swifter.apply(apply_shop_on_market, axis=1)
df_train_x

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_20,item_name_tfidf_21,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,7,6,0,-1,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.578850,0.0,0.0,0.0,1,13,0,-1,0.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.618795,0.0,0.0,0.0,4,57,0,-1,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.468373,0.0,0.0,0.0,1,19,0,-1,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,7,6,0,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,9,4,11,0,34.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,10,44,2,0,34.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,9,4,34,0,34.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,1.0,0.000000,0.0,0.0,0.0,7,6,11,0,34.0


In [18]:
'''
drop_list = df_train_x.loc[df_train_x.shop_on_market < 0].index
df_train_x.drop(drop_list, inplace=True)
df_train_x
'''

'\ndrop_list = df_train_x.loc[df_train_x.shop_on_market < 0].index\ndf_train_x.drop(drop_list, inplace=True)\ndf_train_x\n'

In [19]:
shop_num = (df_train.shop_id.max() + 1)
shop_no_sale = [-1]*date_num*shop_num
all_shops = df_train_x.shop_id.sort_values().unique()

for date in range(1, 35):    
    print("date {}".format(date))
    date_sales = df_train[df_train.date_block_num == (date - 1)].reset_index(drop=True)
    date_shops = date_sales.shop_id.unique()   
    for shop in all_shops:
        if shop not in date_shops:
            if (shop_no_sale[(date - 1) * shop_num + shop] != -1):
                shop_no_sale[date * shop_num + shop] = shop_no_sale[(date - 1) * shop_num + shop] + 1
        else:
            shop_no_sale[date * shop_num + shop] = 0

def apply_shop_no_sale(row):
    return shop_no_sale[int(row.date_block_num) * shop_num + int(row.shop_id)]

df_train_x['shop_no_sale'] = df_train_x.swifter.apply(apply_shop_no_sale, axis=1).astype(np.int8)
df_train_x

date 1
date 2
date 3
date 4
date 5
date 6
date 7
date 8
date 9
date 10
date 11
date 12
date 13
date 14
date 15
date 16
date 17
date 18
date 19
date 20
date 21
date 22
date 23
date 24
date 25
date 26
date 27
date 28
date 29
date 30
date 31
date 32
date 33
date 34


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_21,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,7,6,0,-1,0.0,-1
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.578850,0.0,0.0,0.0,1,13,0,-1,0.0,-1
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.618795,0.0,0.0,0.0,4,57,0,-1,0.0,-1
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.468373,0.0,0.0,0.0,1,19,0,-1,0.0,-1
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,7,6,0,-1,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,9,4,11,0,34.0,0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,10,44,2,0,34.0,0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,9,4,34,0,34.0,0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,7,6,11,0,34.0,0


In [20]:
shop_item_first_sale = [34]*shop_num*item_num
all_shops = df_train.shop_id.sort_values().unique()
all_items = df_train.item_id.sort_values().unique()

for shop in all_shops:
    print("shop {}".format(shop))
    for item in all_items:
        temp = df_train[(df_train.shop_id == shop) & (df_train.item_id == item)]
        if len(temp) > 0:
            shop_item_first_sale[shop*item_num+item] = temp.date_block_num.min()

shop 2
shop 3
shop 4
shop 5
shop 6
shop 7
shop 8
shop 9
shop 10
shop 12
shop 13
shop 14
shop 15
shop 16
shop 17
shop 18
shop 19
shop 20
shop 21
shop 22
shop 23
shop 24
shop 25
shop 26
shop 27
shop 28
shop 29
shop 30
shop 31
shop 32
shop 33
shop 34
shop 35
shop 36
shop 37
shop 38
shop 39
shop 40
shop 41
shop 42
shop 43
shop 44
shop 45
shop 46
shop 47
shop 48
shop 49
shop 50
shop 51
shop 52
shop 53
shop 54
shop 55
shop 56
shop 57
shop 58
shop 59


In [21]:
def apply_shop_item_on_market(row):
    if shop_item_first_sale[int(row.shop_id)*item_num+int(row.item_id)] <= row.date_block_num:
        return row.date_block_num - shop_item_first_sale[int(row.shop_id)*item_num+int(row.item_id)]
    else:
        return -1
    
df_train_x['shop_item_on_market'] = df_train_x.swifter.apply(apply_shop_item_on_market, axis=1).astype(np.int8)
df_train_x

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,7,6,0,-1,0.0,-1,-1
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,1,13,0,-1,0.0,-1,0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,4,57,0,-1,0.0,-1,-1
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,0.0,1,19,0,-1,0.0,-1,-1
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,7,6,0,-1,0.0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,9,4,11,0,34.0,0,11
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,10,44,2,0,34.0,0,0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,9,4,34,0,34.0,0,34
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,7,6,11,0,34.0,0,0


In [22]:
shop_item_no_sale = [-1]*date_num*item_num*shop_num
all_shops = df_train_x.shop_id.sort_values().unique()
all_items = df_train_x.item_id.sort_values().unique()

for date in range(1, 35): 
    print("date {}".format(date))
    date_sales = df_train[df_train.date_block_num == (date - 1)].reset_index(drop=True)
    date_shops = date_sales.shop_id.unique()
    date_items = date_sales.item_id.unique()
    for shop in all_shops:
        for item in all_items:
            if ((shop not in date_shops) or (item not in date_items)):
                if (shop_item_no_sale[(date-1)*shop_num*item_num+shop*item_num+item] != -1):
                    shop_item_no_sale[date*shop_num*item_num+shop*item_num+item] = \
                        shop_item_no_sale[(date-1)*shop_num*item_num+shop*item_num+item] + 1
            else:
                shop_item_no_sale[date*shop_num*item_num+shop*item_num+item] = 0 

def apply_shop_item_no_sale(row):
    return shop_item_no_sale[int(row.date_block_num) * shop_num * item_num + \
            int(row.shop_id)*item_num + int(row.item_id)]

df_train_x['shop_item_no_sale'] = df_train_x.swifter.apply(apply_shop_item_no_sale, axis=1).astype(np.int8)
df_train_x

date 1
date 2
date 3
date 4
date 5
date 6
date 7
date 8
date 9
date 10
date 11
date 12
date 13
date 14
date 15
date 16
date 17
date 18
date 19
date 20
date 21
date 22
date 23
date 24
date 25
date 26
date 27
date 28
date 29
date 30
date 31
date 32
date 33
date 34


Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_23,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,7,6,0,-1,0.0,-1,-1,-1
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.0,1,13,0,-1,0.0,-1,0,-1
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,4,57,0,-1,0.0,-1,-1,-1
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,1,19,0,-1,0.0,-1,-1,-1
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,7,6,0,-1,0.0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,9,4,11,0,34.0,0,11,0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,10,44,2,0,34.0,0,0,0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,9,4,34,0,34.0,0,34,0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,7,6,11,0,34.0,0,0,0


In [30]:
sales = df_train.groupby(['date_block_num', 'item_id'], as_index=False).agg({'item_price':['mean']})
sales.columns = ['date_block_num', 'item_id'] + ['date_item_price']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'item_id'], how = 'left')
del sales

df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,item_name_tfidf_24,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,7,6,0,-1,0.0,-1,-1,-1,1.000000
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,1,13,0,-1,0.0,-1,0,-1,16.714286
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,4,57,0,-1,0.0,-1,-1,-1,8.000000
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,1,19,0,-1,0.0,-1,-1,-1,16.833333
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,7,6,0,-1,0.0,-1,-1,-1,4.831111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,9,4,11,0,34.0,0,11,0,NaN
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,10,44,2,0,34.0,0,0,0,NaN
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,9,4,34,0,34.0,0,34,0,NaN
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,7,6,11,0,34.0,0,0,0,NaN


In [31]:
sales = df_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'item_price':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'item_id'] + ['date_shop_item_price']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id', 'item_id'], how = 'left')
del sales

df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,1.000000,NaN
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,1,13,0,-1,0.0,-1,0,-1,16.714286,17.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,4,57,0,-1,0.0,-1,-1,-1,8.000000,NaN
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,1,19,0,-1,0.0,-1,-1,-1,16.833333,NaN
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,4.831111,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,11,0,34.0,0,11,0,NaN,NaN
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,10,44,2,0,34.0,0,0,0,NaN,NaN
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,34,0,34.0,0,34,0,NaN,NaN
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,7,6,11,0,34.0,0,0,0,NaN,NaN


In [25]:
'''
date_item_price = [-1]*35*item_num
all_items = df_train.item_id.sort_values().unique()

for date in range(34):
    for item in all_items:
        df_date_item_price = df_train[(df_train.date_block_num <= date) & (df_train.item_id == item)]
        if len(df_date_item_price) > 0:
            date_item_price[date*item_num+item] = df_date_item_price.item_price.values[-1]
'''

In [32]:
'''
def apply_date_item_price(row):
    if pd.isnull(row.date_item_price):
        return date_item_price[int(row.date_block_num)*item_num+int(row.item_id)]

df_train_x['date_item_price'] = df_train_x.swifter.apply(apply_date_item_price, axis=1)
'''
df_train_x['date_item_price'] = df_train_x['date_item_price'].fillna(-1).astype(np.int8)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,1,NaN
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,1,13,0,-1,0.0,-1,0,-1,16,17.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,4,57,0,-1,0.0,-1,-1,-1,8,NaN
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,1,19,0,-1,0.0,-1,-1,-1,16,NaN
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,11,0,34.0,0,11,0,-1,NaN
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,10,44,2,0,34.0,0,0,0,-1,NaN
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,34,0,34.0,0,34,0,-1,NaN
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,7,6,11,0,34.0,0,0,0,-1,NaN


In [33]:
'''
def apply_date_shop_item_price(row):
    if pd.isnull(row.date_shop_item_price):
        return date_item_price[int(row.date_block_num)*item_num+int(row.item_id)]

df_train_x['date_shop_item_price'] = df_train_x.swifter.apply(apply_date_shop_item_price, axis=1)
'''
df_train_x['date_shop_item_price'] = df_train_x['date_shop_item_price'].fillna(-1).astype(np.int8)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,cat_type,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,1,-1
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,1,13,0,-1,0.0,-1,0,-1,16,17
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,4,57,0,-1,0.0,-1,-1,-1,8,-1
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,1,19,0,-1,0.0,-1,-1,-1,16,-1
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,7,6,0,-1,0.0,-1,-1,-1,4,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,11,0,34.0,0,11,0,-1,-1
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,10,44,2,0,34.0,0,0,0,-1,-1
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,9,4,34,0,34.0,0,34,0,-1,-1
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,7,6,11,0,34.0,0,0,0,-1,-1


In [34]:
sales = df_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({'date_shop_item_cnt':['first']})
sales.columns = ['date_block_num', 'shop_id', 'item_id'] + ['date_shop_item_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id', 'item_id'], how = 'left')
df_train_x['date_shop_item_cnt'] = df_train_x['date_shop_item_cnt'].fillna(0).clip(0,20).astype(np.float16)
del sales

df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,sub_type,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price,date_shop_item_cnt
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,6,0,-1,0.0,-1,-1,-1,1,-1,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,13,0,-1,0.0,-1,0,-1,16,17,1.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,57,0,-1,0.0,-1,-1,-1,8,-1,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,19,0,-1,0.0,-1,-1,-1,16,-1,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,6,0,-1,0.0,-1,-1,-1,4,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,4,11,0,34.0,0,11,0,-1,-1,0.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,44,2,0,34.0,0,0,0,-1,-1,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,4,34,0,34.0,0,34,0,-1,-1,0.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,6,11,0,34.0,0,0,0,-1,-1,0.0


In [37]:
revenue = df_train.groupby(['date_block_num', 'shop_id'], as_index=False).agg({'date_shop_revenue':['first']})
revenue.columns = ['date_block_num', 'shop_id'] + ['date_shop_revenue']

df_train_x = pd.merge(df_train_x, revenue, on=['date_block_num', 'shop_id'], how = 'left')
df_train_x['date_shop_revenue'] = df_train_x['date_shop_revenue'].fillna(0).astype(np.float16)
del revenue

df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,time_on_market,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price,date_shop_item_cnt,date_shop_revenue
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0,-1,0.0,-1,-1,-1,1,-1,0.0,1085.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0,-1,0.0,-1,0,-1,16,17,1.0,1085.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0,-1,0.0,-1,-1,-1,8,-1,0.0,1085.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0,-1,0.0,-1,-1,-1,16,-1,0.0,1085.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0,-1,0.0,-1,-1,-1,4,-1,0.0,1085.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,11,0,34.0,0,11,0,-1,-1,0.0,0.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,2,0,34.0,0,0,0,-1,-1,0.0,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,34,0,34.0,0,34,0,-1,-1,0.0,0.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,11,0,34.0,0,0,0,-1,-1,0.0,0.0


In [38]:
revenue = df_train.groupby(['date_block_num', 'shop_id','item_category_id'], as_index=False).agg({'date_shop_cat_revenue':['first']})
revenue.columns = ['date_block_num', 'shop_id','item_category_id'] + ['date_shop_cat_revenue']

df_train_x = pd.merge(df_train_x, revenue, on=['date_block_num', 'shop_id', 'item_category_id'], how = 'left')
df_train_x['date_shop_cat_revenue'] = df_train_x['date_shop_cat_revenue'].fillna(0).astype(np.float16)
del revenue

df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,time_no_sale,shop_on_market,shop_no_sale,shop_item_on_market,shop_item_no_sale,date_item_price,date_shop_item_price,date_shop_item_cnt,date_shop_revenue,date_shop_cat_revenue
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,-1,0.0,-1,-1,-1,1,-1,0.0,1085.0,38.53125
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,-1,0.0,-1,0,-1,16,17,1.0,1085.0,170.25000
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,-1,0.0,-1,-1,-1,8,-1,0.0,1085.0,92.18750
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,-1,0.0,-1,-1,-1,16,-1,0.0,1085.0,67.37500
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,-1,0.0,-1,-1,-1,4,-1,0.0,1085.0,38.53125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0,34.0,0,11,0,-1,-1,0.0,0.0,0.00000
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0,34.0,0,0,0,-1,-1,0.0,0.0,0.00000
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0,34.0,0,34,0,-1,-1,0.0,0.0,0.00000
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0,34.0,0,0,0,-1,-1,0.0,0.0,0.00000


In [39]:
gc.collect()

21

In [40]:
df_train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11126966 entries, 0 to 11126965
Data columns (total 44 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date_block_num         int8   
 1   shop_id                int8   
 2   item_id                int16  
 3   shop_city              int8   
 4   shop_type              int8   
 5   item_category_id       int8   
 6   item_name_tfidf_0      float64
 7   item_name_tfidf_1      float64
 8   item_name_tfidf_2      float64
 9   item_name_tfidf_3      float64
 10  item_name_tfidf_4      float64
 11  item_name_tfidf_5      float64
 12  item_name_tfidf_6      float64
 13  item_name_tfidf_7      float64
 14  item_name_tfidf_8      float64
 15  item_name_tfidf_9      float64
 16  item_name_tfidf_10     float64
 17  item_name_tfidf_11     float64
 18  item_name_tfidf_12     float64
 19  item_name_tfidf_13     float64
 20  item_name_tfidf_14     float64
 21  item_name_tfidf_15     float64
 22  item_name_tfidf_

#### 5.3 Combo Features

In [41]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_id'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_id'] + ['avg_date_item_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'item_id'], how = 'left')
df_train_x['avg_date_item_cnt'] = (df_train_x['avg_date_item_cnt'].fillna(0).astype(np.float16))

del sales

In [42]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_id', 'shop_city'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_id', 'shop_city'] + ['avg_date_city_item_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'item_id', 'shop_city'], how = 'left')
df_train_x['avg_date_city_item_cnt'] = (df_train_x['avg_date_city_item_cnt'].fillna(0).astype(np.float16))

del sales

In [43]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_id', 'shop_type'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_id', 'shop_type'] + ['avg_date_shop_type_item_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'item_id', 'shop_type'], how = 'left')
df_train_x['avg_date_shop_type_item_cnt'] = (df_train_x['avg_date_shop_type_item_cnt'].fillna(0).astype(np.float16))

del sales

In [44]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_category_id'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_category_id'] + ['avg_date_cat_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'item_category_id'], how = 'left')
df_train_x['avg_date_cat_cnt'] = (df_train_x['avg_date_cat_cnt'].fillna(0).astype(np.float16))

del sales

In [45]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num','shop_id','item_category_id'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'item_category_id'] + ['avg_date_shop_cat_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id','item_category_id'], how = 'left')
df_train_x['avg_date_shop_cat_cnt'] = (df_train_x['avg_date_shop_cat_cnt'].fillna(0).astype(np.float16))

del sales

In [46]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'sub_type'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'sub_type'] + ['avg_date_sub_type_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'sub_type'], how = 'left')
df_train_x['avg_date_sub_type_cnt'] = (df_train_x['avg_date_sub_type_cnt'].fillna(0).astype(np.float16))

del sales

In [47]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'shop_id', 'sub_type'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'sub_type'] + ['avg_date_shop_sub_type_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id', 'sub_type'], how = 'left')
df_train_x['avg_date_shop_sub_type_cnt'] = (df_train_x['avg_date_shop_sub_type_cnt'].fillna(0).astype(np.float16))

del sales

In [48]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'cat_type'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'cat_type'] + ['avg_date_cat_type_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'cat_type'], how = 'left')
df_train_x['avg_date_cat_type_cnt'] = (df_train_x['avg_date_cat_type_cnt'].fillna(0).astype(np.float16))

del sales

In [49]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'shop_id', 'cat_type'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'cat_type'] + ['avg_date_shop_cat_type_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id', 'cat_type'], how = 'left')
df_train_x['avg_date_shop_cat_type_cnt'] = (df_train_x['avg_date_shop_cat_type_cnt'].fillna(0).astype(np.float16))

del sales

In [50]:
# price dependent sales.

sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_id', 'date_shop_item_price'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_id', 'date_shop_item_price', 'avg_date_item_price_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num','item_id', 'date_shop_item_price'], how = 'left')
df_train_x['avg_date_item_price_cnt'] = (df_train_x['avg_date_item_price_cnt'].fillna(0).clip(0,20).astype(np.float16))

del sales

In [51]:
# price dependent sales.

sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'shop_id', 'item_id', 'date_shop_item_price'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'item_id', 'date_shop_item_price', 'avg_date_shop_item_price_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num','shop_id', 'item_id', 'date_shop_item_price'], how = 'left')
df_train_x['avg_date_shop_item_price_cnt'] = (df_train_x['avg_date_shop_item_price_cnt'].fillna(0).clip(0,20).astype(np.float16))

del sales

In [52]:

sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'item_category_id', 'date_shop_item_price'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'item_category_id', 'date_shop_item_price', 'avg_date_cat_price_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num','item_category_id','date_shop_item_price'], how = 'left')
df_train_x['avg_date_cat_price_cnt'] = (df_train_x['avg_date_cat_price_cnt'].fillna(0).clip(0,20).astype(np.float16))

del sales

In [54]:
sales = df_train_x[df_train_x.time_on_market >= 0].groupby(['date_block_num', 'shop_id', 'item_category_id', 'date_shop_item_price'], as_index=False).agg({'date_shop_item_cnt':['mean']})
sales.columns = ['date_block_num', 'shop_id', 'item_category_id', 'date_shop_item_price', 'avg_date_shop_cat_price_cnt']

df_train_x = pd.merge(df_train_x, sales, on=['date_block_num', 'shop_id', 'item_category_id', 'date_shop_item_price'], how = 'left')
df_train_x['avg_date_shop_cat_price_cnt'] = (df_train_x['avg_date_shop_cat_price_cnt'].fillna(0).clip(0,20).astype(np.float16))

del sales

#### 5.4 Lag Features

In [55]:
def add_lag_features(df, lags, index, col):
    tmp = df.groupby(index, as_index=False).agg({col:['first']})
    
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = index + [col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        
        df = pd.merge(df, shifted, on=index, how='left')
        
        del shifted
    
    del tmp
    
    return df

In [56]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_id'], 'date_shop_item_cnt')
df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_shop_sub_type_cnt,avg_date_cat_type_cnt,avg_date_shop_cat_type_cnt,avg_date_item_price_cnt,avg_date_shop_item_price_cnt,avg_date_cat_price_cnt,avg_date_shop_cat_price_cnt,date_shop_item_cnt_lag_1,date_shop_item_cnt_lag_2,date_shop_item_cnt_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.073120,0.271729,0.073853,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.515625,0.727539,0.427002,1.0,1.0,3.240234,1.222656,0.0,0.0,0.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.537109,1.138672,0.505859,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.260986,0.727539,0.427002,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.073120,0.271729,0.073853,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [57]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_id'], 'date_item_price')
df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_item_price_cnt,avg_date_shop_item_price_cnt,avg_date_cat_price_cnt,avg_date_shop_cat_price_cnt,date_shop_item_cnt_lag_1,date_shop_item_cnt_lag_2,date_shop_item_cnt_lag_3,date_item_price_lag_1,date_item_price_lag_2,date_item_price_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,1.0,1.0,3.240234,1.222656,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,1.0,1.0,1.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,14.0,13.0,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,3.0,3.0,3.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,1.0


In [58]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_id'], 'date_shop_item_price')
df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_shop_cat_price_cnt,date_shop_item_cnt_lag_1,date_shop_item_cnt_lag_2,date_shop_item_cnt_lag_3,date_item_price_lag_1,date_item_price_lag_2,date_item_price_lag_3,date_shop_item_price_lag_1,date_shop_item_price_lag_2,date_shop_item_price_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,1.222656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,1.0,0.0,0.0,1.0,1.0,1.0,1.0,-1.0,-1.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,14.0,13.0,0.0,-1.0,-1.0,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,3.0,3.0,3.0,-1.0,-1.0,-1.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,1.0,1.0,1.0,-1.0,-1.0,-1.0


In [59]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_id'], 'avg_date_item_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_id', 'shop_city'], 'avg_date_city_item_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_id', 'shop_type'], 'avg_date_shop_type_item_cnt')

df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,date_shop_item_price_lag_3,avg_date_item_cnt_lag_1,avg_date_item_cnt_lag_2,avg_date_item_cnt_lag_3,avg_date_city_item_cnt_lag_1,avg_date_city_item_cnt_lag_2,avg_date_city_item_cnt_lag_3,avg_date_shop_type_item_cnt_lag_1,avg_date_shop_type_item_cnt_lag_2,avg_date_shop_type_item_cnt_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,-1.0,0.045441,0.023254,0.071411,0.5,0.0,0.0,0.090881,0.000000,0.043488
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.022720,0.069763,0.000000,0.0,0.0,0.0,0.000000,0.083313,0.000000
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,-1.0,0.113647,0.069763,0.095215,0.0,0.5,0.0,0.136353,0.083313,0.086975
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,-1.0,0.045441,0.069763,0.166626,0.0,0.0,0.0,0.090881,0.083313,0.217407


In [60]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_category_id'], 'avg_date_cat_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_category_id'],'avg_date_shop_cat_cnt')

df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'sub_type'], 'avg_date_sub_type_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'sub_type'], 'avg_date_shop_sub_type_cnt')

df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'cat_type'], 'avg_date_cat_type_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'cat_type'], 'avg_date_shop_cat_type_cnt')

df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_sub_type_cnt_lag_3,avg_date_shop_sub_type_cnt_lag_1,avg_date_shop_sub_type_cnt_lag_2,avg_date_shop_sub_type_cnt_lag_3,avg_date_cat_type_cnt_lag_1,avg_date_cat_type_cnt_lag_2,avg_date_cat_type_cnt_lag_3,avg_date_shop_cat_type_cnt_lag_1,avg_date_shop_cat_type_cnt_lag_2,avg_date_shop_cat_type_cnt_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.217285,0.126831,0.074524,0.106079,0.146973,0.139038,0.163818,0.089294,0.053802,0.071167
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.265137,0.094482,0.117126,0.219971,0.236450,0.216553,0.266113,0.106201,0.112732,0.141357
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.217285,0.126831,0.074524,0.106079,0.146973,0.139038,0.163818,0.089294,0.053802,0.071167
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.242188,0.083740,0.074097,0.111633,0.222900,0.196045,0.213501,0.097046,0.072571,0.111023


In [61]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_id', 'date_item_price'], 'avg_date_item_price_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_id', 'date_item_price'], 'avg_date_shop_item_price_cnt')

df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'item_category_id', 'date_shop_item_price'], 'avg_date_cat_price_cnt')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_category_id', 'date_shop_item_price'], 'avg_date_shop_cat_price_cnt')

df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_item_price_cnt_lag_3,avg_date_shop_item_price_cnt_lag_1,avg_date_shop_item_price_cnt_lag_2,avg_date_shop_item_price_cnt_lag_3,avg_date_cat_price_cnt_lag_1,avg_date_cat_price_cnt_lag_2,avg_date_cat_price_cnt_lag_3,avg_date_shop_cat_price_cnt_lag_1,avg_date_shop_cat_price_cnt_lag_2,avg_date_shop_cat_price_cnt_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id'], 'date_shop_revenue')
df_train_x = add_lag_features(df_train_x, [1,2,3], ['date_block_num', 'shop_id', 'item_category_id'], 'date_shop_cat_revenue')

df_train_x.fillna(0, inplace=True)
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_cat_price_cnt_lag_3,avg_date_shop_cat_price_cnt_lag_1,avg_date_shop_cat_price_cnt_lag_2,avg_date_shop_cat_price_cnt_lag_3,date_shop_revenue_lag_1,date_shop_revenue_lag_2,date_shop_revenue_lag_3,date_shop_cat_revenue_lag_1,date_shop_cat_revenue_lag_2,date_shop_cat_revenue_lag_3
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,854.5,866.0,655.0,28.453125,15.976562,22.234375
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,854.5,866.0,655.0,13.890625,12.437500,24.453125
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,854.5,866.0,655.0,28.453125,15.976562,22.234375
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,854.5,866.0,655.0,21.687500,16.343750,22.078125


In [ ]:
df_train_x.drop('date_item_price', axis = 1, inplace = True)
df_train_x.drop('date_shop_item_price', axis = 1, inplace = True)

df_train_x.drop('avg_date_item_price_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_item_price_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_cat_price_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_cat_price_cnt', axis = 1, inplace = True)

In [71]:
df_train_x.drop('date_shop_revenue', axis = 1, inplace = True)
df_train_x.drop('date_shop_cat_revenue', axis = 1, inplace = True)

df_train_x.drop('avg_date_item_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_city_item_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_type_item_cnt', axis = 1, inplace = True)

df_train_x.drop('avg_date_cat_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_cat_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_sub_type_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_sub_type_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_cat_type_cnt', axis = 1, inplace = True)
df_train_x.drop('avg_date_shop_cat_type_cnt', axis = 1, inplace = True)

In [72]:
df_train_x["month"] = df_train_x["date_block_num"] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
df_train_x["days"] = df_train_x["month"].map(days).astype(np.int8)

#### 5.4 Save Data

In [73]:
df_train_x.columns

Index(['date_block_num', 'shop_id', 'item_id', 'shop_city', 'shop_type',
       'item_category_id', 'item_name_tfidf_0', 'item_name_tfidf_1',
       'item_name_tfidf_2', 'item_name_tfidf_3', 'item_name_tfidf_4',
       'item_name_tfidf_5', 'item_name_tfidf_6', 'item_name_tfidf_7',
       'item_name_tfidf_8', 'item_name_tfidf_9', 'item_name_tfidf_10',
       'item_name_tfidf_11', 'item_name_tfidf_12', 'item_name_tfidf_13',
       'item_name_tfidf_14', 'item_name_tfidf_15', 'item_name_tfidf_16',
       'item_name_tfidf_17', 'item_name_tfidf_18', 'item_name_tfidf_19',
       'item_name_tfidf_20', 'item_name_tfidf_21', 'item_name_tfidf_22',
       'item_name_tfidf_23', 'item_name_tfidf_24', 'cat_type', 'sub_type',
       'time_on_market', 'time_no_sale', 'shop_on_market', 'shop_no_sale',
       'shop_item_on_market', 'shop_item_no_sale', 'date_shop_item_cnt',
       'date_shop_item_cnt_lag_1', 'date_shop_item_cnt_lag_2',
       'date_shop_item_cnt_lag_3', 'date_item_price_lag_1',
       'd

In [74]:
df_train_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11126966 entries, 0 to 11126965
Data columns (total 96 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   date_block_num                      int8   
 1   shop_id                             int8   
 2   item_id                             int16  
 3   shop_city                           int8   
 4   shop_type                           int8   
 5   item_category_id                    int8   
 6   item_name_tfidf_0                   float64
 7   item_name_tfidf_1                   float64
 8   item_name_tfidf_2                   float64
 9   item_name_tfidf_3                   float64
 10  item_name_tfidf_4                   float64
 11  item_name_tfidf_5                   float64
 12  item_name_tfidf_6                   float64
 13  item_name_tfidf_7                   float64
 14  item_name_tfidf_8                   float64
 15  item_name_tfidf_9                   float64
 16

In [75]:
df_train_x.to_pickle('v30_data.pkl')

gc.collect();

In [76]:
print("complete!")

complete!


In [77]:
df_train_x

,date_block_num,shop_id,item_id,shop_city,shop_type,item_category_id,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,...,avg_date_shop_cat_price_cnt_lag_2,avg_date_shop_cat_price_cnt_lag_3,date_shop_revenue_lag_1,date_shop_revenue_lag_2,date_shop_revenue_lag_3,date_shop_cat_revenue_lag_1,date_shop_cat_revenue_lag_2,date_shop_cat_revenue_lag_3,month,days
0,0,2,19,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,31
1,0,2,27,1,7,19,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,31
2,0,2,28,1,7,30,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,31
3,0,2,29,1,7,23,0.0,0.588259,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,31
4,0,2,32,1,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11126961,34,45,18454,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,854.5,866.0,655.0,28.453125,15.976562,22.234375,10,30
11126962,34,45,16188,22,7,64,0.0,0.000000,0.0,0.000000,...,0.0,0.0,854.5,866.0,655.0,13.890625,12.437500,24.453125,10,30
11126963,34,45,15757,22,7,55,0.0,0.000000,0.0,0.000000,...,0.0,0.0,854.5,866.0,655.0,28.453125,15.976562,22.234375,10,30
11126964,34,45,19648,22,7,40,0.0,0.000000,0.0,0.000000,...,0.0,0.0,854.5,866.0,655.0,21.687500,16.343750,22.078125,10,30
